In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("SparkAdvancedOps")\
        .master("spark://spark-master:7077").getOrCreate() 
# df = spark.read.format("csv")\
#           .option("header", "true")\
#           .option("inferSchema", "true")\
#           .load("/home/jovyan/data/airline3")\
#           .coalesce(5)


df = spark.read \
  .format("bigquery") \
  .load("prefab-clover-330908.airline_2.correct_input_csv_batch")   


df1 = df
df2 = df

from pyspark.sql.functions import count
sub_total_dest = df1.groupBy("Month", "Week_number", "Dest").agg(count("NASDelay").alias("sub_total_NASDelay")).na.fill(0).collect()
df_sub_total_dest = spark.createDataFrame(sub_total_dest)

count_dest = df1.groupBy("Month", "Week_number", "Dest").count().collect()
df_count_dest = spark.createDataFrame(count_dest)

mergedf_dest = df_sub_total_dest.join(df_count_dest, ["Month", "Week_number", "Dest"]).collect()
df_mergedf_dest = spark.createDataFrame(mergedf_dest)
df_mergedf_dest = df_mergedf_dest.withColumnRenamed("Dest", "City").withColumnRenamed("count", "count1").withColumnRenamed("sub_total_NASDelay", "s_t_NASDelay1")

sub_total_origin = df2.groupBy("Month", "Week_number", "Origin").agg(count("NASDelay").alias("sub_total_NASDelay")).na.fill(0).collect()
df_sub_total_origin = spark.createDataFrame(sub_total_origin)

count_org = df2.groupBy("Month", "Week_number", "Origin").count().collect()
df_count_org = spark.createDataFrame(count_org)

mergedf_org = df_sub_total_origin.join(df_count_org, ["Month", "Week_number", "Origin"]).collect()
df_mergedf_org = spark.createDataFrame(mergedf_org)
df_mergedf_org = df_mergedf_org.withColumnRenamed("Origin", "City").withColumnRenamed("count", "count2").withColumnRenamed("sub_total_NASDelay", "s_t_NASDelay2")

total_merge = df_mergedf_dest.join(df_mergedf_org, ["Month", "Week_number", "City"])

from pyspark.sql.functions import lit, col
df_total = total_merge.withColumn("total_nr_flights", lit(col("count1") + col("count2")))
drop_flights = df_total.where((col("total_nr_flights") >= 10) ).select("*") # use only the cities that got 10 or more flights so that only
# airports with atleast some traffic are selected

df_total_dr = drop_flights.drop("count1", "count2")

df_total_2 = df_total_dr.withColumn("total_NAS_delay", lit(col("s_t_NASDelay1") + col("s_t_NASDelay2")))

df_total_2_dr = df_total_2.drop("s_t_NASDelay1", "s_t_NASDelay2")

final_output = df_total_2_dr.withColumn("avg_NASDelay", col("total_NAS_delay")/ col("total_nr_flights"))

minutes_to_seconds = final_output.withColumn("avg_NASDelay_sec", round(col("avg_NASDelay") * 60))
#minutes_to_seconds.show()

final_output_2 = minutes_to_seconds.drop("total_nr_flights", "total_NAS_delay", "avg_NASDelay")

from pyspark.sql.functions import *
from pyspark.sql import Row, Window

windowdesc = Window.partitionBy(col("Week_number")).orderBy(col("avg_NASDelay_sec").desc())

airline_merged_windowed = final_output_2.withColumn("rank_desc", dense_rank().over(windowdesc))
#airline_merged_windowed.show()

airline2 = airline_merged_windowed.where((col("rank_desc") <= 10) ).select("*")
#airline2.show()

airline2_table2 = airline2.select("Month", "Week_number", "City", "avg_NASDelay_sec", "rank_desc").withColumnRenamed("rank_desc", "Worst_performing_cities")

worst_performing_output = airline2_table2


conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")


bucket = "airplane_chris_ass2_batch"    
spark.conf.set('temporaryGcsBucket', bucket)


# Saving the data to BigQuery
worst_performing_output.write.format('bigquery') \
  .option('table', 'prefab-clover-330908.airline_2.worst_performing') \
  .mode("append") \
  .save()

+---+-----+
| gt|count|
+---+-----+
+---+-----+

+---+-----+
| gt|count|
+---+-----+
+---+-----+

+---+-----+
| gt|count|
+---+-----+
+---+-----+

+---+-----+
| gt|count|
+---+-----+
+---+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|10452|
|       sit|12310|
|     stand|11385|
|      walk|13256|
|      bike|10797|
|stairsdown| 9365|
|      null|10447|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|20905|
|       sit|24620|
|     stand|22770|
|      walk|26512|
|      bike|21594|
|stairsdown|18729|
|      null|20894|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|31357|
|       sit|36929|
|     stand|34154|
|      walk|39768|
|      bike|32390|
|stairsdown|28094|
|      null|31343|
+----------+-----+

+----------+-----+
|        gt|count|
+----------+-----+
|  stairsup|31357|
|       sit|36929|
|     stand|34154|
|      walk|39768|
|      bike|32390|
|stairsdown|28094|
|  

In [6]:
# Stop the spark context
spark.stop()